In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from datetime import timedelta
from timeit import default_timer as timer
from math import sqrt

from random import choices
from scipy.linalg import null_space
from scipy.optimize import minimize

from PIL import ImageDraw

In [2]:
img_l = np.asanyarray(Image.open("../myL.jpg").convert("RGB")).astype(np.int)
img_r = np.asanyarray(Image.open("../myL.jpg").convert("RGB")).astype(np.int)
img_l = img_l[::10,::10]
img_r = img_r[::10,::10]

img_shift = np.load("../lab1/img_shift.npy")

bestF = np.load("../lab2/bestF.npy")

In [3]:
img_l.shape

(195, 260, 3)

In [4]:
scale = 10
pixel_size = 1.12e-3
pixel_size *= scale
focal = 3
width = img_r.shape[1]
height = img_r.shape[0]

In [5]:
K = np.array([[focal/pixel_size, 0, width/2],
              [0, focal/pixel_size, height/2],
              [0, 0, 1]])
K

array([[267.85714286,   0.        , 130.        ],
       [  0.        , 267.85714286,  97.5       ],
       [  0.        ,   0.        ,   1.        ]])

In [6]:
bestF

array([[ 8.55854749e-15,  6.91289031e-13, -3.02955607e-01],
       [-6.45401668e-13,  4.18780136e-14, -6.44751676e-01],
       [ 3.02955607e-01,  6.52519769e-01,  2.36150012e+00]])

In [7]:
E = np.dot(K.T, np.dot(bestF, K))
E_bak = E.copy()
#E = E/1000
np.round(E,2)

array([[   0.  ,    0.  ,  -81.15],
       [  -0.  ,    0.  , -172.7 ],
       [  81.15,  174.78,    3.12]])

In [8]:
U, D, T = np.linalg.svd(E)
print(*map(lambda x: round(x,4), D))
D[:2] = D[:2].mean()
E = np.linalg.multi_dot([U, np.diag(D), T])
print(*map(lambda x: round(x,4), D))

193.5875 189.9431 0.0
191.7653 191.7653 0.0


In [9]:
2*np.linalg.multi_dot([E, E.T, E]) - E*np.trace(np.dot(E, E.T))

array([[ 9.09494702e-11,  9.24046617e-10,  0.00000000e+00],
       [ 9.24046617e-10,  1.62981451e-09,  0.00000000e+00],
       [ 9.31322575e-10,  1.86264515e-09, -2.05182005e-09]])

In [10]:
np.round(2*np.linalg.multi_dot([E, E.T, E]) - E*np.trace(np.dot(E, E.T)),2)

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0., -0.]])

## 2

In [11]:
U, S, V = np.linalg.svd(E)

In [12]:
W = np.dot(U, np.array([[0,1,0],
                        [-1,0,0],
                        [0,0,1]]))

In [13]:
R = np.linalg.det(W)*np.linalg.det(V)*np.dot(W, V)

In [14]:
np.round(R,3)

array([[ 1.   ,  0.005,  0.003],
       [-0.005,  1.   ,  0.007],
       [-0.003, -0.007,  1.   ]])

In [15]:
v3 = V.T[:,2]
v3

array([ 9.07008968e-01, -4.21111306e-01, -2.50520167e-10])

In [16]:
c = v3*S[0]*np.linalg.det(W)
np.round(c,3)

array([173.933, -80.755,  -0.   ])

In [17]:
c_vec = np.array([[0,-c[2],c[1]],
                  [c[2],0,-c[0]],
                  [-c[1],c[0],0]])

In [18]:
np.round(E - np.dot(R,c_vec), 10)

array([[-0., -0.,  0.],
       [-0., -0.,  0.],
       [-0., -0., -0.]])

In [19]:
np.linalg.det(R)

1.0000000000000042

In [20]:
np.linalg.det(c_vec)

-1.1874480816995176e-19